In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [4]:
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False) 

In [22]:
df = pd.read_csv(r'C:\Users\clair\OneDrive\Desktop\Data Analysis\marketing_data.csv')

In [24]:
balances = df['BALANCE']

In [28]:
# Calculate statistics
average_balance = balances.mean()

In [29]:
# Calculate statistics
min_balance = balances.min()

In [31]:
# Calculate statistics
max_balance = balances.max()

In [32]:
print("Average Balance:", average_balance) 
print("Minimum Balance:", min_balance)
print("Maximum Balance:", max_balance)

Average Balance: 1564.4748276781006
Minimum Balance: 0.0
Maximum Balance: 19043.13856


In [34]:
# Get summary stats on all columns
print(df.describe())

            BALANCE  BALANCE_FREQUENCY     PURCHASES  ONEOFF_PURCHASES  \
count   8950.000000        8950.000000   8950.000000       8950.000000   
mean    1564.474828           0.877271   1003.204834        592.437371   
std     2081.531879           0.236904   2136.634782       1659.887917   
min        0.000000           0.000000      0.000000          0.000000   
25%      128.281915           0.888889     39.635000          0.000000   
50%      873.385231           1.000000    361.280000         38.000000   
75%     2054.140036           1.000000   1110.130000        577.405000   
max    19043.138560           1.000000  49039.570000      40761.250000   

       INSTALLMENTS_PURCHASES  CASH_ADVANCE  PURCHASES_FREQUENCY  \
count             8950.000000   8950.000000          8950.000000   
mean               411.067645    978.871112             0.490351   
std                904.338115   2097.163877             0.401371   
min                  0.000000      0.000000             0.000

In [53]:
#Features for Customer with Maximum One-Off Balance
try:
    max_purchases_row = df.loc[df['ONEOFF_PURCHASES'].idxmax()]
    cust_id = max_purchases_row['CUST_ID'] 
except KeyError:   
    print("CUST_ID column not present in the dataframe")
    cust_id = "Not available"

print("Customer ID:", max_purchases_row['CUST_ID'])  
print("One Off Purchases:", max_purchases_row['ONEOFF_PURCHASES']) 

print("Other Features:")
print(max_purchases_row[['BALANCE', 'PURCHASES', 'CREDIT_LIMIT']])


Customer ID: C10574
One Off Purchases: 40761.25
Other Features:
BALANCE         11547.52001
PURCHASES          49039.57
CREDIT_LIMIT        22500.0
Name: 550, dtype: object


In [55]:
# Features of Customer with Maximum Cash Advance Transactions
max_advance_row = df.loc[df['CASH_ADVANCE'].idxmax()]

# print customer ID 
print("Customer ID:", max_advance_row['CUST_ID'])

# print their maximum cash advance amount
print("Maximum Cash Advance:", max_advance_row['CASH_ADVANCE'])  

# number of cash advance transactions made
print("Cash Advance Transactions:", max_advance_row['CASH_ADVANCE_TRX'])

# cash advance frequency for customer  
print("Cash Advance Frequency:", max_advance_row['CASH_ADVANCE_FREQUENCY'])

# full payment frequency 
print("Full Payment Frequency:", max_advance_row['PRC_FULL_PAYMENT'])

Customer ID: C12226
Maximum Cash Advance: 47137.21176
Cash Advance Transactions: 123
Cash Advance Frequency: 1.0
Full Payment Frequency: 0.0


In [56]:
# Checking for Missing Data
print(df.isnull().sum())

# Check percentage of rows with missing values
print(df.isnull().mean()*100)

# Dropping rows having missing values
df_no_null = df.dropna()
print("Number of Rows after dropping null values:") 
print(len(df_no_null))

CUST_ID                               0
BALANCE                               0
BALANCE_FREQUENCY                     0
PURCHASES                             0
ONEOFF_PURCHASES                      0
INSTALLMENTS_PURCHASES                0
CASH_ADVANCE                          0
PURCHASES_FREQUENCY                   0
ONEOFF_PURCHASES_FREQUENCY            0
PURCHASES_INSTALLMENTS_FREQUENCY      0
CASH_ADVANCE_FREQUENCY                0
CASH_ADVANCE_TRX                      0
PURCHASES_TRX                         0
CREDIT_LIMIT                          1
PAYMENTS                              0
MINIMUM_PAYMENTS                    313
PRC_FULL_PAYMENT                      0
TENURE                                0
dtype: int64
CUST_ID                             0.000000
BALANCE                             0.000000
BALANCE_FREQUENCY                   0.000000
PURCHASES                           0.000000
ONEOFF_PURCHASES                    0.000000
INSTALLMENTS_PURCHASES              0.0000

In [57]:
# Check number of duplicate rows
print("Duplicate Rows: ")
print(df.duplicated().sum())

print("Samples:")
print(df[df.duplicated()])  

df_no_dups = df.drop_duplicates() 

# Compare row count before and after
print("Number of Rows Before:")  
print(len(df))
print("Number of Rows after Removing Duplicates:")  
print(len(df_no_dups))

Duplicate Rows: 
0
Samples:
Empty DataFrame
Columns: [CUST_ID, BALANCE, BALANCE_FREQUENCY, PURCHASES, ONEOFF_PURCHASES, INSTALLMENTS_PURCHASES, CASH_ADVANCE, PURCHASES_FREQUENCY, ONEOFF_PURCHASES_FREQUENCY, PURCHASES_INSTALLMENTS_FREQUENCY, CASH_ADVANCE_FREQUENCY, CASH_ADVANCE_TRX, PURCHASES_TRX, CREDIT_LIMIT, PAYMENTS, MINIMUM_PAYMENTS, PRC_FULL_PAYMENT, TENURE]
Index: []
Number of Rows Before:
8950
Number of Rows after Removing Duplicates:
8950


In [58]:
## Dropping Customer ID Column
# Verify CUST_ID column is present 
print("Columns before dropping:\n", df.columns)

# Drop CUST_ID column
df = df.drop('CUST_ID', axis=1)

# Verify column removed
print("Columns after dropping:\n", df.columns)

Columns before dropping:
 Index(['CUST_ID', 'BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES',
       'ONEOFF_PURCHASES', 'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE',
       'PURCHASES_FREQUENCY', 'ONEOFF_PURCHASES_FREQUENCY',
       'PURCHASES_INSTALLMENTS_FREQUENCY', 'CASH_ADVANCE_FREQUENCY',
       'CASH_ADVANCE_TRX', 'PURCHASES_TRX', 'CREDIT_LIMIT', 'PAYMENTS',
       'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT', 'TENURE'],
      dtype='object')
Columns after dropping:
 Index(['BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES', 'ONEOFF_PURCHASES',
       'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE', 'PURCHASES_FREQUENCY',
       'ONEOFF_PURCHASES_FREQUENCY', 'PURCHASES_INSTALLMENTS_FREQUENCY',
       'CASH_ADVANCE_FREQUENCY', 'CASH_ADVANCE_TRX', 'PURCHASES_TRX',
       'CREDIT_LIMIT', 'PAYMENTS', 'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT',
       'TENURE'],
      dtype='object')


In [59]:
## Correlation Matrix
# Get correlation matrix 
corr_matrix = df.corr()

# Print full correlation matrix
print(corr_matrix)

# Print summary statistics  
print(corr_matrix.describe())

                                   BALANCE  BALANCE_FREQUENCY  PURCHASES  \
BALANCE                           1.000000           0.322412   0.181261   
BALANCE_FREQUENCY                 0.322412           1.000000   0.133674   
PURCHASES                         0.181261           0.133674   1.000000   
ONEOFF_PURCHASES                  0.164350           0.104323   0.916845   
INSTALLMENTS_PURCHASES            0.126469           0.124292   0.679896   
CASH_ADVANCE                      0.496692           0.099388  -0.051474   
PURCHASES_FREQUENCY              -0.077944           0.229715   0.393017   
ONEOFF_PURCHASES_FREQUENCY        0.073166           0.202415   0.498430   
PURCHASES_INSTALLMENTS_FREQUENCY -0.063186           0.176079   0.315567   
CASH_ADVANCE_FREQUENCY            0.449218           0.191873  -0.120143   
CASH_ADVANCE_TRX                  0.385152           0.141555  -0.067175   
PURCHASES_TRX                     0.154338           0.189626   0.689561   
CREDIT_LIMIT

In [61]:
# Removing Rows with Missing Values
df = df.dropna()
kmeans.fit(df)

KMeans(n_clusters=1)

In [62]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
df_imputed = imputer.fit_transform(df)

kmeans.fit(df_imputed)

KMeans(n_clusters=1)

In [70]:
# Check column names
print(df.columns)

target_column = 'PURCHASES' # Example column 

# Check if column exists
if target_column not in df.columns:
  print(f"{target_column} does not exist! Using default")
  target_column = "BALANCE" # fallback column

from sklearn.ensemble import HistGradientBoostingRegressor

y = df['PURCHASES']  
X = df.drop('PURCHASES', axis=1)

model = HistGradientBoostingRegressor().fit(X, y)
  


Index(['BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES', 'ONEOFF_PURCHASES',
       'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE', 'PURCHASES_FREQUENCY',
       'ONEOFF_PURCHASES_FREQUENCY', 'PURCHASES_INSTALLMENTS_FREQUENCY',
       'CASH_ADVANCE_FREQUENCY', 'CASH_ADVANCE_TRX', 'PURCHASES_TRX',
       'CREDIT_LIMIT', 'PAYMENTS', 'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT',
       'TENURE'],
      dtype='object')
